In [1]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import csv
import os
from copy import deepcopy

from datetime import datetime

In [2]:
folder_name = "TS_03.이상행동_07.전도"

root = "/data/ephemeral/home/datasets/abnormal/" + folder_name + "/"


header = [
    "Filename",
    "Frame",
    "ID",
    "X",
    "Y",
    "Width",
    "Height",
    "Keypoint_0",
    "Keypoint_1",
    "Keypoint_2",
    "Keypoint_3",
    "Keypoint_4",
    "Keypoint_5",
    "Keypoint_6",
    "Keypoint_7",
    "Keypoint_8",
    "Keypoint_9",
    "Keypoint_10",
    "Keypoint_11",
    "Keypoint_12",
    "Keypoint_13",
    "Keypoint_14",
    "Keypoint_15",
    "Keypoint_16",
    "Keypoint_17",
    "Keypoint_18",
    "Keypoint_19",
    "Keypoint_20",
    "Keypoint_21",
    "Keypoint_22",
    "Keypoint_23",
    "Keypoint_24",
    "Keypoint_25",
    "Keypoint_26",
    "Keypoint_27",
    "Keypoint_28",
    "Keypoint_29",
    "Keypoint_30",
    "Keypoint_31",
    "Keypoint_32",
    "Keypoint_33",
]

In [3]:
with open(f"{folder_name}.csv", "w") as c_file:
    writer = csv.writer(c_file, delimiter=",")

    writer.writerow(header)

In [4]:
file_list = os.listdir(root)
print(f"==>> file_list: {file_list}")
file_list.sort()
print(f"==>> file_list: {file_list}")

==>> file_list: ['C_3_7_30_BU_SMB_09-02_13-39-50_CD_RGB_DF2_F3.mp4', 'C_3_7_43_BU_SMC_10-14_09-41-04_CB_RGB_DF2_M2.mp4', 'C_3_7_13_BU_SMB_09-02_13-01-15_CA_RGB_DF2_M2.mp4', 'C_3_7_22_BU_SYB_10-04_10-31-24_CB_RGB_DF2_M3.mp4', 'C_3_7_28_BU_SYB_10-04_10-15-34_CA_RGB_DF2_F3.mp4', 'C_3_7_25_BU_SYA_10-06_11-01-43_CA_RGB_DF2_M3.mp4', 'C_3_7_11_BU_SMB_09-01_12-43-41_CC_RGB_DF2_M2.mp4', 'C_3_7_31_BU_SYB_10-04_10-20-35_CA_RGB_DF2_F3.mp4', 'C_3_7_3_BU_SMA_08-28_13-22-56_CD_RGB_DF2_M1.mp4', 'C_3_7_31_BU_SMC_08-07_15-09-31_CF_RGB_DF2_M1.mp4', 'C_3_7_13_BU_SMA_09-07_14-33-29_CD_RGB_DF2_M2.mp4', 'C_3_7_40_BU_SMC_10-16_10-20-46_CC_RGB_DF2_F1.mp4', 'C_3_7_20_BU_SMA_09-07_15-29-14_CD_RGB_DF2_F2.mp4', 'C_3_7_7_BU_SMB_09-01_12-33-26_CA_RGB_DF2_M2.mp4', 'C_3_7_4_BU_SMA_08-30_13-28-42_CA_RGB_DF2_F1.mp4', 'C_3_7_19_BU_SMA_09-07_15-26-04_CC_RGB_DF2_F2.mp4', 'C_3_7_8_BU_SYB_09-28_11-36-42_CB_RGB_DF2_M2.mp4', 'C_3_7_21_BU_SMB_09-02_15-23-06_CD_RGB_DF2_M3.mp4', 'C_3_7_3_BU_SMA_08-30_13-25-19_CA_RGB_DF2_M1.mp4', 

In [5]:
# Load the YOLOv8 model
model = YOLO("yolov8n-pose.pt")

# Define the standard frame size (change these values as needed)
standard_width = 640
standard_height = 480

In [6]:
# # test
# file_list = file_list[:5]
# print(f"==>> file_list: {file_list}")

In [7]:


id_count = 0

for file_name in file_list:
    path = root + file_name

    time_start = datetime.now()

    print(f"{file_name} feature extracting starts")

    cap = cv2.VideoCapture(path)

    # Loop through the video frames
    frame_count = 0

    # Store the track history
    track_history = defaultdict(lambda: [])

    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        frame_count += 1  # Increment frame count

        if success:
            frame = cv2.resize(frame, (standard_width, standard_height))

            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True, verbose=False)

            if results[0].boxes is not None:  # Check if there are results and boxes
                # Get the boxes
                # boxes = results[0].boxes.xywh.cpu()

                if results[0].boxes.id is not None:
                    # If 'int' attribute exists (there are detections), get the track IDs
                    track_ids = results[0].boxes.id.int().cpu().tolist()

                    for i, box in zip(range(0, len(track_ids)), results[0].boxes.xywhn.cpu()):
                        keypoints = results[0].keypoints.xyn[i].cpu().numpy().flatten().tolist()
                        box_list = box.numpy().flatten().tolist()
                        if type(box_list) == 'float' or type(keypoints) == 'float':
                            print(f"==>> box_list: {box_list}")
                            print(f"==>> keypoints: {keypoints}")
                        box_and_keypoints = box_list + keypoints
                        track_history[track_ids[i]].append([[frame_count], deepcopy(box_and_keypoints)])
        else:
            # Break the loop if the end of the video is reached
            break

    with open(f"{folder_name}.csv", "a") as c_file:
        writer = csv.writer(c_file, delimiter=",")
        for key in track_history.keys():
            for f_count, b_and_k in track_history[key]:
                row = [file_name] + f_count + [id_count+key] + b_and_k

                writer.writerow(row)

    id_count = id_count + len(track_history.keys())

    cap.release()

    time_end = datetime.now()
    total_time = time_end - time_start
    total_time = str(total_time).split(".")[0]

    print(f"{file_name} feature extracting ended. Elapsed time: {total_time}")

# cv2.destroyAllWindows()

C_3_7_10_BU_DYA_08-23_13-47-37_CA_RGB_DF2_M2.mp4 feature extracting starts


C_3_7_10_BU_DYA_08-23_13-47-37_CA_RGB_DF2_M2.mp4 feature extracting ended. Elapsed time: 0:00:09
C_3_7_10_BU_DYA_08-23_13-47-40_CB_RGB_DF2_M2.mp4 feature extracting starts
C_3_7_10_BU_DYA_08-23_13-47-40_CB_RGB_DF2_M2.mp4 feature extracting ended. Elapsed time: 0:00:05
C_3_7_10_BU_DYA_08-23_13-47-40_CC_RGB_DF2_M2.mp4 feature extracting starts
C_3_7_10_BU_DYA_08-23_13-47-40_CC_RGB_DF2_M2.mp4 feature extracting ended. Elapsed time: 0:00:05
C_3_7_10_BU_SMA_09-07_14-20-21_CA_RGB_DF2_M2.mp4 feature extracting starts
C_3_7_10_BU_SMA_09-07_14-20-21_CA_RGB_DF2_M2.mp4 feature extracting ended. Elapsed time: 0:00:05
C_3_7_10_BU_SMA_09-07_14-20-21_CB_RGB_DF2_M2.mp4 feature extracting starts
C_3_7_10_BU_SMA_09-07_14-20-21_CB_RGB_DF2_M2.mp4 feature extracting ended. Elapsed time: 0:00:05
C_3_7_10_BU_SMA_09-07_14-20-21_CC_RGB_DF2_M2.mp4 feature extracting starts
C_3_7_10_BU_SMA_09-07_14-20-21_CC_RGB_DF2_M2.mp4 feature extracting ended. Elapsed time: 0:00:05
C_3_7_10_BU_SMA_09-07_14-20-21_CD_RGB_DF2_M

In [ ]:
# folder_name = "VS_02.구매행동_06.비교"

# root = "/data/ephemeral/home/datasets/normal/" + folder_name + "/"

# with open(f"{folder_name}.csv", "w") as c_file:
#     writer = csv.writer(c_file, delimiter=",")

#     writer.writerow(header)


# file_list = os.listdir(root)
# print(f"==>> file_list: {file_list}")
# file_list.sort()
# print(f"==>> file_list: {file_list}")

==>> file_list: ['C_2_6_63_BU_SMB_10-15_11-26-48_CA_RGB_DF1_M4_M4.mp4', 'C_2_6_56_BU_SMB_10-15_16-18-47_CB_RGB_DF1_F3_F3.mp4', 'C_2_6_63_BU_SYB_10-15_10-35-02_CA_RGB_DF1_M4_M4.mp4', 'C_2_6_57_BU_DYB_10-20_14-39-22_CD_RGB_DF1_M4_M4.mp4', 'C_2_6_57_BU_DYB_10-20_14-39-22_CA_RGB_DF1_M4_M4.mp4', 'C_2_6_55_BU_DYB_10-23_15-27-36_CB_RGB_DF1_F3_F3.mp4', 'C_2_6_59_BU_SMB_10-15_16-21-33_CD_RGB_DF1_F3_F3.mp4', 'C_2_6_63_BU_SYB_10-15_10-35-02_CD_RGB_DF1_M4_M4.mp4', 'C_2_6_60_BU_SYB_10-15_10-32-02_CA_RGB_DF1_M4_M4.mp4', 'C_2_6_59_BU_SMB_10-15_16-21-33_CC_RGB_DF1_F3_F3.mp4', 'C_2_6_61_BU_DYB_10-20_14-51-00_CB_RGB_DF1_M4_M4.mp4', 'C_2_6_56_BU_SMB_10-15_16-18-47_CA_RGB_DF1_F3_F3.mp4', 'C_2_6_56_BU_SYA_09-15_13-45-22_CA_RGB_DF1_F3_F3.mp4', 'C_2_6_57_BU_DYB_10-20_14-39-22_CC_RGB_DF1_M4_M4.mp4', 'C_2_6_65_BU_SMB_10-15_11-28-21_CA_RGB_DF1_F4_F4.mp4', 'C_2_6_61_BU_SMB_10-15_11-25-05_CD_RGB_DF1_M4_M4.mp4', 'C_2_6_57_BU_SYA_09-15_13-46-52_CB_RGB_DF1_F3_F3.mp4', 'C_2_6_55_BU_DYB_10-23_15-27-36_CA_RGB_DF1_F3_F3

In [ ]:
# # Loop through the video frames
# frame_count = 0

# id_count = 0

# for file_name in file_list:
#     path = root + file_name

#     time_start = datetime.now()

#     print(f"{file_name} feature extracting starts")

#     cap = cv2.VideoCapture(path)

#     # Store the track history
#     track_history = defaultdict(lambda: [])

#     while cap.isOpened():
#         # Read a frame from the video
#         success, frame = cap.read()

#         frame_count += 1  # Increment frame count

#         if success:
#             frame = cv2.resize(frame, (standard_width, standard_height))

#             # Run YOLOv8 tracking on the frame, persisting tracks between frames
#             results = model.track(frame, persist=True, verbose=False)

#             if results[0].boxes is not None:  # Check if there are results and boxes
#                 # Get the boxes
#                 # boxes = results[0].boxes.xywh.cpu()

#                 if results[0].boxes.id is not None:
#                     # If 'int' attribute exists (there are detections), get the track IDs
#                     track_ids = results[0].boxes.id.int().cpu().tolist()

#                     for i, box in zip(range(0, len(track_ids)), results[0].boxes.xywhn.cpu()):
#                         keypoints = results[0].keypoints.xyn[i].cpu().numpy().flatten().tolist()
#                         box_list = box.numpy().flatten().tolist()
#                         if type(box_list) == "float" or type(keypoints) == "float":
#                             print(f"==>> box_list: {box_list}")
#                             print(f"==>> keypoints: {keypoints}")
#                         box_and_keypoints = box_list + keypoints
#                         track_history[track_ids[i]].append([[frame_count], deepcopy(box_and_keypoints)])
#         else:
#             # Break the loop if the end of the video is reached
#             break

#     with open(f"{folder_name}.csv", "a") as c_file:
#         writer = csv.writer(c_file, delimiter=",")
#         for key in track_history.keys():
#             for f_count, b_and_k in track_history[key]:
#                 row = f_count + [id_count + key] + b_and_k

#                 writer.writerow(row)

#     id_count = id_count + len(track_history.keys())

#     cap.release()

#     time_end = datetime.now()
#     total_time = time_end - time_start
#     total_time = str(total_time).split(".")[0]

#     print(f"{file_name} feature extracting ended. Elapsed time: {total_time}")

C_2_6_55_BU_DYB_10-23_15-27-36_CA_RGB_DF1_F3_F3.mp4 feature extracting starts
C_2_6_55_BU_DYB_10-23_15-27-36_CA_RGB_DF1_F3_F3.mp4 feature extracting ended. Elapsed time: 0:00:19
C_2_6_55_BU_DYB_10-23_15-27-36_CB_RGB_DF1_F3_F3.mp4 feature extracting starts
C_2_6_55_BU_DYB_10-23_15-27-36_CB_RGB_DF1_F3_F3.mp4 feature extracting ended. Elapsed time: 0:00:20
C_2_6_55_BU_DYB_10-23_15-27-36_CC_RGB_DF1_F3_F3.mp4 feature extracting starts
C_2_6_55_BU_DYB_10-23_15-27-36_CC_RGB_DF1_F3_F3.mp4 feature extracting ended. Elapsed time: 0:00:17
C_2_6_55_BU_DYB_10-23_15-27-36_CD_RGB_DF1_F3_F3.mp4 feature extracting starts
C_2_6_55_BU_DYB_10-23_15-27-36_CD_RGB_DF1_F3_F3.mp4 feature extracting ended. Elapsed time: 0:00:16
C_2_6_56_BU_DYA_07-28_11-43-36_CC_RGB_DF1_F4_F4.mp4 feature extracting starts
C_2_6_56_BU_DYA_07-28_11-43-36_CC_RGB_DF1_F4_F4.mp4 feature extracting ended. Elapsed time: 0:00:18
C_2_6_56_BU_DYA_07-28_11-43-37_CB_RGB_DF1_F4_F4.mp4 feature extracting starts
C_2_6_56_BU_DYA_07-28_11-43-37_C